In [1]:
using JuMP
#using GLPKMathProgInterface
using CPLEX

# Modelo simple

\begin{align}
\text{minimize} \qquad & x+y \\
 \text{subject to} \quad \quad & x+y \geq 1 \\
 \qquad \qquad & x \geq 0, y \geq 0 \\
 \qquad \qquad & x,y \in \mathbb{R}
\end{align}



In [2]:
#m = Model(solver = GLPKSolverMIP())
#m = Model(solver = GLPKSolverLP())

#creamos un modelo
m = Model(solver = CplexSolver())

#y sus variables
@variable(m, x >= 0 )
@variable(m, y >= 0 )

#función objetivo a minimizar
@objective(m, Min, x + y )

#restricción
@constraint(m, x + y >= 1.0 )

#escribimos el modelo
print(m)

#resolvemos
status = solve(m)

#mostramos resultados
println("Status: ",status," Objective value: ", getobjectivevalue(m))
println("x = ", getvalue(x))
println("y = ", getvalue(y))

Min x + y
Subject to
 x + y ≥ 1
 x ≥ 0
 y ≥ 0
Tried aggregator 1 time.
LP Presolve eliminated 1 rows and 2 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)
Status: Optimal Objective value: 1.0
x = 0.0
y = 1.0
CPLEX Error  3003: Not a mixed-integer problem.


# Un programa lineal en formato estándar

Un PL en formato estándar se expresa como:

$$
\begin{align}
& \text{min} && c^T x \\
& \text{subject to} && A x = b \\
&                   && x \succeq 0 \\
&                   && x \in \mathbb{R}^n
\end{align}
$$


In [3]:
#Datos
#----------

#INPUT DATA
#----------

c=[2500; 4000; 0; 0; 0] 
b=[200; 100; 750]
A=[1 0 1 0 0; 0 1 0 1 0; 3 5 0 0 1]
m, n = size(A) # m = filas n = columnas

(3,5)

In [4]:
modelo = Model(solver = CplexSolver())
@variable(modelo, x[1:n] >= 0) # Models x >=0
for i in 1:m # filas
    @constraint(modelo, sum(A[i, j]*x[j] for j in 1:n) == b[i]) # i-ésima restricción 
end 
@objective(modelo,Min,sum(c[i]*x[i] for i in 1:n))
status = solve(modelo) # solves the model  

Tried aggregator 1 time.
LP Presolve eliminated 3 rows and 5 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)
CPLEX Error  3003: Not a mixed-integer problem.


:Optimal

# Un modelo algo más complejo

Obviamente, el modelo anterior no tiene mayor dificultad. Alternativamente podríamos intentar resolver algún modelo con algo más de historia (como por ejemplo el Uncapacitated Lot Sizing Problem).

\begin{align}
\text{minimize} \qquad & \sum_{t\in T} c_s s_t+ \sum_{t\in T} c_K y_t \\
 \text{subject to} \quad \quad & s_{t-1} + x_t = s_t + d_t \\
 \qquad \qquad & x_t \leq M y_t \\
 \qquad \qquad & s_0 = 0 \\
 \qquad \qquad & x_t, s_t \in \mathbb{R}^+ \forall t\in T\\
 \qquad \qquad & y_t \in \{0,1\}
\end{align}

$c_s$ y $c_K$ son los costes de inventario (unidad por periodo) y de realizar un pedido en un periodo y $d_t$ la demanda de cada periodo. El conjunto de periodos es $T$. $x_t$ es la cantidad que se recibe en el periodo $t$, mientras que $y_t$ equivale a si se realiza un pedido en el periodo $t$ o no y $s_t$ es el inventario al final del periodo $t$.

El objetivo minimiza la suma de costos de inventario y pedido (el costo por unidad no se tiene en cuenta porque la solución óptima siempre compra exactamente la demanda, $\sum_{t\in T} d_t$.

La primera restricción asegura el cumplimiento de la demanda de cada periodo. La segunda restricción asegura que sólo se compran unidades si se ha realizado un pedido ($M$ es un valor lo suficientemente grande, en este caso basta con $\sum_{t\in T} d_t$). La tercera restricción inicializa el inventario y el resto de restricciones definen el problema.

In [5]:
T=10
d=[5; 10; 7; 11; 13; 14; 2; 8; 17; 21]
cK=50
cs=2
M=sum(d)

108

In [6]:
modelo=Model(solver = CplexSolver())
@variable(modelo, x[1:T]>= 0)
@variable(modelo, y[1:T],Bin)
@variable(modelo, s[0:T]>=0)

#función objetivo a minimizar
@objective(modelo, Min, sum(cs*s[t] for t in 1:T)+sum(cK*y[t] for t in 1:T))

for t in 1:T
    @constraint(modelo,s[t-1]+x[t]==d[t]+s[t])
end
for t in 1:T
    @constraint(modelo,x[t]<=M*y[t])
end
@constraint(modelo,s[0]==0)

#escribimos el modelo
print(modelo)

#resolvemos
status = solve(modelo)

#mostramos resultados
println("**** Status: ",status," Objective value: ", getobjectivevalue(modelo))
println("**** x = ", getvalue(x))
println("**** y = ", getvalue(y))
println("**** s = ", getvalue(s))

Min 2 s[1] + 2 s[2] + 2 s[3] + 2 s[4] + 2 s[5] + 2 s[6] + 2 s[7] + 2 s[8] + 2 s[9] + 2 s[10] + 50 y[1] + 50 y[2] + 50 y[3] + 50 y[4] + 50 y[5] + 50 y[6] + 50 y[7] + 50 y[8] + 50 y[9] + 50 y[10]
Subject to
 s[0] + x[1] - s[1] = 5
 s[1] + x[2] - s[2] = 10
 s[2] + x[3] - s[3] = 7
 s[3] + x[4] - s[4] = 11
 s[4] + x[5] - s[5] = 13
 s[5] + x[6] - s[6] = 14
 s[6] + x[7] - s[7] = 2
 s[7] + x[8] - s[8] = 8
 s[8] + x[9] - s[9] = 17
 s[9] + x[10] - s[10] = 21
 x[1] - 108 y[1] ≤ 0
 x[2] - 108 y[2] ≤ 0
 x[3] - 108 y[3] ≤ 0
 x[4] - 108 y[4] ≤ 0
 x[5] - 108 y[5] ≤ 0
 x[6] - 108 y[6] ≤ 0
 x[7] - 108 y[7] ≤ 0
 x[8] - 108 y[8] ≤ 0
 x[9] - 108 y[9] ≤ 0
 x[10] - 108 y[10] ≤ 0
 s[0] = 0
 x[i] ≥ 0 ∀ i ∈ {1,2,…,9,10}
 y[i] ∈ {0,1} ∀ i ∈ {1,2,…,9,10}
 s[i] ≥ 0 ∀ i ∈ {0,1,…,9,10}
Found incumbent of value 378.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 3 rows and 3 columns.
Reduced MIP has 18 rows, 28 columns, and 45 nonzeros.
Reduced MIP has 9 binaries, 0 generals, 0 SO

# Set Covering Problem

Veamos un ejemplo un poco más complejo en que es importante considerar que la mayoría de la matriz A está formada por ceros.

El modelo es:

\begin{align}
\text{minimize} \qquad & \sum_{j=1}^{n} c_j x_j \\
 \text{subject to} \quad \quad & \sum_{i=1}^{m} a_{ij} x_j \geq 1 \forall i\in 1,\cdots,m \\
 \qquad \qquad & x_j \in \{0,1\} \forall j\in 1,\cdots,n
\end{align}

Primero vamos a bajar un archivo de datos del mismo 

In [7]:
run(`wget -O scp41.txt http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/scp41.txt`)

--2017-09-26 17:07:30--  http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/scp41.txt
Resolving people.brunel.ac.uk (people.brunel.ac.uk)... 134.83.117.116
Connecting to people.brunel.ac.uk (people.brunel.ac.uk)|134.83.117.116|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20562 (20K) [text/plain]
Saving to: ‘scp41.txt’

     0K .......... ..........                                 100% 83.8K=0.2s

2017-09-26 17:07:33 (83.8 KB/s) - ‘scp41.txt’ saved [20562/20562]



In [8]:
run(`cat scp41.txt`)

 200 1000 
 1 1 1 1 1 1 1 1 1 1 1 1 
 2 2 2 2 2 2 2 2 2 2 2 2 
 2 2 2 3 3 3 3 3 3 3 3 3 
 3 3 3 3 3 3 4 4 4 4 4 4 
 4 4 4 4 4 4 4 4 5 5 5 5 
 5 5 5 5 5 6 6 6 6 6 6 6 
 6 6 6 6 7 7 7 7 7 7 7 7 
 7 7 7 8 8 8 8 8 8 8 8 8 
 8 9 9 9 9 9 9 9 9 10 10 10 
 10 10 10 10 10 10 11 11 11 11 11 11 
 11 12 12 12 12 12 12 12 12 12 12 12 
 12 12 12 13 13 13 13 13 13 13 13 14 
 14 14 14 14 14 14 14 14 14 14 14 14 
 14 15 15 15 15 15 15 15 15 15 15 15 
 15 15 15 15 15 16 16 16 16 16 16 17 
 17 17 17 17 18 18 18 18 18 18 18 18 
 18 18 18 18 18 19 19 19 19 19 19 19 
 20 20 20 20 20 20 20 20 20 20 20 20 
 20 21 21 21 21 21 21 21 21 22 22 22 
 22 22 22 22 22 22 22 23 23 23 23 23 
 23 23 23 23 23 23 24 24 24 24 24 24 
 24 24 24 24 24 25 25 25 25 25 25 25 
 25 26 26 26 26 26 26 26 26 27 27 27 
 27 27 27 27 27 27 27 27 27 27 28 28 
 28 28 28 28 28 29 29 29 29 29 29 29 
 29 30 30 30 30 30 31 31 31 31 31 31 
 31 31 31 31 32 32 32 32 32 32 32 32 
 32 32 32 33 33 33 33 33 33 33 33 33 
 33 34 34 34 34 34 34 34 35 35

La estructura de datos es algo confusa. Primero tenemos una línea con el número de restricciones (200) y en número de variables (1000). Los siguientes 1000 valores corresponden a los costos de las variables en la función objetivo y después hay la descripción de las 200 restricciones con esta forma (copio la primera)

 17 
 91 214 230 289 351 416 488 491 518 567 720 721 
 735 753 768 928 990 
 
 Que significa que hay 17 no ceros y que las variables 91, ..., 990 adoptan valor en la restriccion.
 
 Como la estructura es un tanto oscura, vamos a tener que crear nuestro propio lector de instancias:

In [9]:
function readFile(filename)
    f=open(filename,"r")
    s=readstring(f)
    close(f)
    s=replace(s,"\n","")
    s=split(s," ")
    nC=parse(Int64,s[2])
    nV=parse(Int64,s[3])
    numCoefFo=0
    numCoef=0
    numRow=0
    c= Float64[]
    I=Int64[]
    J=Int64[]
    V=Float64[]
    for i in 4:length(s)
        if s[i]!= ""
            if numCoefFo < nV
                push!(c,parse(Float64,s[i]))
                numCoefFo += 1
            else
                if numCoef == 0
                    numRow += 1
                    numCoef = parse(Int64,s[i])
                else
                    numCoef -= 1
                    push!(I,numRow)
                    push!(J,parse(Int64,s[i]))
                    push!(V,1.0)
                end
            end
        end
    end
    A=sparse(I,J,V)
    return nC,nV,c,A #,c,A
end

readFile (generic function with 1 method)

In [10]:
constraints,variables,c,A = readFile("scp41.txt")

#println("\n\n",constraints," ",variables)
#println("\n\n",c)
#println("\n\n",A)

(200,1000,[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0  …  100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0],

	[18  ,    1]  =  1.0
	[32  ,    1]  =  1.0
	[75  ,    1]  =  1.0
	[76  ,    1]  =  1.0
	[107 ,    1]  =  1.0
	[190 ,    1]  =  1.0
	[196 ,    1]  =  1.0
	[199 ,    1]  =  1.0
	[3   ,    2]  =  1.0
	[4   ,    2]  =  1.0
	⋮
	[102 ,  996]  =  1.0
	[184 ,  996]  =  1.0
	[50  ,  997]  =  1.0
	[111 ,  997]  =  1.0
	[184 ,  997]  =  1.0
	[81  ,  998]  =  1.0
	[98  ,  998]  =  1.0
	[7   ,  999]  =  1.0
	[197 ,  999]  =  1.0
	[55  , 1000]  =  1.0
	[183 , 1000]  =  1.0)

In [11]:
scp=Model(solver = CplexSolver())
@variable(scp, x[1:variables],Bin)

#función objetivo a minimizar
@objective(scp, Min, sum(c[j]*x[j] for j in 1:variables))

for i in 1:constraints
    @constraint(scp,sum(A[i,j]*x[j] for j in 1:variables) >=1 )
end

#escribimos el modelo
print(scp)

#resolvemos
status = solve(scp)

#mostramos resultados
println("**** Status: ",status," Objective value: ", getobjectivevalue(scp))
println("**** x = ", getvalue(x))



Min x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + 2 x[13] + 2 x[14] + 2 x[15] + 2 x[16] + 2 x[17] + 2 x[18] + 2 x[19] + 2 x[20] + 2 x[21] + 2 x[22] + 2 x[23] + 2 x[24] + 2 x[25] + 2 x[26] + 2 x[27] + 3 x[28] + 3 x[29] + 3 x[30] + 3 x[31] + 3 x[32] + 3 x[33] + 3 x[34] + 3 x[35] + 3 x[36] + 3 x[37] + 3 x[38] + 3 x[39] + 3 x[40] + 3 x[41] + 3 x[42] + 4 x[43] + 4 x[44] + 4 x[45] + 4 x[46] + 4 x[47] + 4 x[48] + 4 x[49] + 4 x[50] + 4 x[51] + 4 x[52] + 4 x[53] + 4 x[54] + 4 x[55] + 4 x[56] + 5 x[57] + 5 x[58] + 5 x[59] + 5 x[60] + 5 x[61] + 5 x[62] + 5 x[63] + 5 x[64] + 5 x[65] + 6 x[66] + 6 x[67] + 6 x[68] + 6 x[69] + 6 x[70] + 6 x[71] + 6 x[72] + 6 x[73] + 6 x[74] + 6 x[75] + 6 x[76] + 7 x[77] + 7 x[78] + 7 x[79] + 7 x[80] + 7 x[81] + 7 x[82] + 7 x[83] + 7 x[84] + 7 x[85] + 7 x[86] + 7 x[87] + 8 x[88] + 8 x[89] + 8 x[90] + 8 x[91] + 8 x[92] + 8 x[93] + 8 x[94] + 8 x[95] + 8 x[96] + 8 x[97] + 9 x[98] + 9 x[99] + 9 x[100] + 9 x[101] + 9 x[102] + 9 x[10

**** Status: Optimal Objective value: 429.0
**** x = [1.0,1.0,1.0,-0.0,1.0,1.0,-0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,-0.0,1.0,1.0,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,1.0,1.0,-0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,-0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-0.0,0.0,1.0,-0.0,0.0,1.0,1.0,1.0,-0.0,0.0,-0.0,1.0,-0.0,1.0,1.0,-0.0,-0.0,1.0,-0.0,1.0,-0.0,1.0,1.0,-0.0,-0.0,1.0,0.0,1.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,0.0,0.0,1.0,-0.0,-0.0,1.0,-0.0,-0.0,0.0,0.0,0.0,1.0,-0.0,-0.0,-0.0,1.0,1.0,1.0,0.0,1.0,-0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,0.0,1.0,1.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,1.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0

Otra alternativa (que en muchas ocasiones es más práctica) es construir la expresión asociada a cada restricción:

In [12]:
scpAlt=Model(solver = CplexSolver())
@variable(scpAlt, xAlt[1:variables],Bin)

#función objetivo a minimizar
Expr = AffExpr()
for j in 1:variables
    push!(Expr, c[j], xAlt[j])
end
@objective(scpAlt, Min, Expr)

for i in 1:constraints
    Expr = AffExpr()
    for j in 1:variables
        push!(Expr,A[i,j],xAlt[j])
    end
    @constraint(scpAlt,Expr >= 1)
end
#for i in 1:constraints
#    @constraint(scp,sum(A[i,j]*x[j] for j in 1:variables) >=1 )
#end

#escribimos el modelo
print(scpAlt)

#resolvemos
status = solve(scpAlt)

#mostramos resultados
println("**** Status: ",status," Objective value: ", getobjectivevalue(scpAlt))
println("**** x = ", getvalue(xAlt))



Min xAlt[1] + xAlt[2] + xAlt[3] + xAlt[4] + xAlt[5] + xAlt[6] + xAlt[7] + xAlt[8] + xAlt[9] + xAlt[10] + xAlt[11] + xAlt[12] + 2 xAlt[13] + 2 xAlt[14] + 2 xAlt[15] + 2 xAlt[16] + 2 xAlt[17] + 2 xAlt[18] + 2 xAlt[19] + 2 xAlt[20] + 2 xAlt[21] + 2 xAlt[22] + 2 xAlt[23] + 2 xAlt[24] + 2 xAlt[25] + 2 xAlt[26] + 2 xAlt[27] + 3 xAlt[28] + 3 xAlt[29] + 3 xAlt[30] + 3 xAlt[31] + 3 xAlt[32] + 3 xAlt[33] + 3 xAlt[34] + 3 xAlt[35] + 3 xAlt[36] + 3 xAlt[37] + 3 xAlt[38] + 3 xAlt[39] + 3 xAlt[40] + 3 xAlt[41] + 3 xAlt[42] + 4 xAlt[43] + 4 xAlt[44] + 4 xAlt[45] + 4 xAlt[46] + 4 xAlt[47] + 4 xAlt[48] + 4 xAlt[49] + 4 xAlt[50] + 4 xAlt[51] + 4 xAlt[52] + 4 xAlt[53] + 4 xAlt[54] + 4 xAlt[55] + 4 xAlt[56] + 5 xAlt[57] + 5 xAlt[58] + 5 xAlt[59] + 5 xAlt[60] + 5 xAlt[61] + 5 xAlt[62] + 5 xAlt[63] + 5 xAlt[64] + 5 xAlt[65] + 6 xAlt[66] + 6 xAlt[67] + 6 xAlt[68] + 6 xAlt[69] + 6 xAlt[70] + 6 xAlt[71] + 6 xAlt[72] + 6 xAlt[73] + 6 xAlt[74] + 6 xAlt[75] + 6 xAlt[76] + 7 xAlt[77] + 7 xAlt[78] + 7 xAlt[79] + 7 

**** Status: Optimal Objective value: 429.0
**** x = [1.0,1.0,1.0,-0.0,1.0,1.0,-0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,-0.0,1.0,1.0,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,1.0,1.0,-0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,-0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-0.0,0.0,1.0,-0.0,0.0,1.0,1.0,1.0,-0.0,0.0,-0.0,1.0,-0.0,1.0,1.0,-0.0,-0.0,1.0,-0.0,1.0,-0.0,1.0,1.0,-0.0,-0.0,1.0,0.0,1.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,0.0,0.0,1.0,-0.0,-0.0,1.0,-0.0,-0.0,0.0,0.0,0.0,1.0,-0.0,-0.0,-0.0,1.0,1.0,1.0,0.0,1.0,-0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,0.0,1.0,1.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,1.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0

# Modificar el comportamiento del solver

Antes de dar por concluída esta sesión convendría ver cómo eliminar esos logs tan largos que ofrece Cplex (entre otros solvers).

La idea es pasar un "flag" al solver (aquí hay una lista de los parámetros de cplex: https://www.ibm.com/support/knowledgecenter/SSSA5P_12.6.1/ilog.odms.cplex.help/CPLEX/Parameters/topics/introListAlpha.html)


In [16]:
#creamos un modelo
m = Model(solver = CplexSolver(CPX_PARAM_SCRIND=0))

#y sus variables
@variable(m, x >= 0 )
@variable(m, y >= 0 )

#función objetivo a minimizar
@objective(m, Min, x + y )

#restricción
@constraint(m, x + y >= 1.0 )

#escribimos el modelo
print(m)

#resolvemos
status = solve(m)

#mostramos resultados
println("Status: ",status," Objective value: ", getobjectivevalue(m))
println("x = ", getvalue(x))
println("y = ", getvalue(y))

Min x + y
Subject to
 x + y ≥ 1
 x ≥ 0
 y ≥ 0
Status: Optimal Objective value: 1.0
x = 0.0
y = 1.0
